<a href="https://colab.research.google.com/github/Gabuchev/Apartments-in-Moscow---The-simplest-recommendation-system/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
import nltk
import re
data = pd.read_csv('/content/Сlean.csv')

In [ ]:
texts=data['description']

In [ ]:
type(texts)

In [ ]:
#Предобработка текста
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")
def token_and_stem(text):
    tokens=[word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens=[]
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems=[stemmer.stem(t) for t in filtered_tokens]
    return stems

In [ ]:
#Стоп слова
stopwords=nltk.corpus.stopwords.words('russian')
#расширяю список стоп слов
stopwords.extend(['что','это','так', 'вот', 'быть','как','в', 'к','на', 'спешите', 'внимание', 'лучшее', 'самое'])

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_featur=200000
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=stopwords,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3))

In [ ]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)